## 本代码在kaggle使用T4成功运行。
### kaggle 链接：https://www.kaggle.com/czy111/llm-for-dealing-with-unstructured-text

In [ ]:
%%time
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
max_seq_length = 4096 
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True 

def init_model(model_name):
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name =model_name,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    return model,tokenizer
model_name='unsloth/Qwen2-7B-Instruct-bnb-4bit'
model,tokenizer=init_model(model_name)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-06-29 14:52:13.403753: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 14:52:13.403867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 14:52:13.706202: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Qwen2 patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
## 用于指令微调的prompt
alpaca_prompt = """下面是描述任务的指令，并配有提供进一步上下文的输入。请按指令要求给出合适的回答。

### 指令:
{}

### 输入:
{}

### 回答:
{}"""

## 保存llm回答，用于后续可视化
def save_llm_respond(respond,save_path):
    with open(save_path,'w') as f:
        f.write(respond)

In [6]:
## 测试文本
input1="""
兴建铁路作为一种官方政策是在新政之前就确立的，对近代中国的铁路建设起到了积极作用。自甲午战争结束到1911年，清王朝共建铁路4936公里，占近代中国所修铁路将近40%，而其中绝大部分是在新政时期建设而成。但主要依靠外资资助修筑的就长达4029公里，约占总长度的81%。但是作为回报，清政府不得不把铁路的管理权、用人权、稽核权、购料权拱手让给外国的借款公司，这就导致了中国利权的严重流失。即便如此，李鸿章在世时还是不无遗憾地说道：“目下经费难筹，必借洋款。”

1903年12月2日，满清政府颁布了《铁路简明章程》，规定“无论华人、洋人”均可向“督抚衙门递呈请办铁路”，而且包括“干路或枝路”，华人投资50万两以上实有成效者，朝廷还要“专折请旨给予优奖”。各省绅商随之掀起规模巨大的收回路权运动，这种行动在朝野上下产生了更大的共鸣。粤汉铁路于1900年承包给美国永兴公司建造，湖南、湖北和广东三省绅士以美国商人违反合同为由，要求收回粤汉铁路筑路权，转而自己修筑，此举得到张之洞的鼎力支持，湖南士绅王先谦从中周旋，终于在1905年8月以赔款675万美元为代价，赎回原先由美国公司发行的股票，废除了与美商签订的建设粤汉铁路的合同。在这个成功案例的鼓舞下，浙江、江苏、山东、直隶、吉林、云南等省收回路权的工作也先后付诸实施，要求赎回沪宁铁路、苏甬杭铁路、广九铁路承办权的社会呼声越来越高，并实际收回了一批筑路权，铁路商办运动进入高潮时期。在此后4年间，全国在15个省创设了18个大型铁路公司，其中17个是商办、官督商办或官商合办的。

清末商办铁路虽然资本匮乏，但是在国人爱国主义的热情鼓舞之下，均设置了严格的受款规定，坚决拒绝外资、不招洋股、不借洋债，故而必须进行大量的民间融资。铁路建设费用极大，一时间又筹不到足够资金，于是政府不得不出面协助，一时“租股、派股”盛极全国各地。湖北咨议局全体会议通过决议，后来又进一步提出拒款方案十项，其中包括对各房主以其两月房租收入购买股票，开设铁路彩票，不用外国技师以节省经费。并进而决定由各府县分担股额，各县从五万元到十万元不等。此外要求湖北全省教育会成员每人每月以薪俸的十分之一购股，由此合计可达四百二十余万元。他们进而估计，湖北各地的商会、军人会、及其他团体，均深受舆论刺激，分担股份不成问题，估计可达二千数百万元。但当时一些社会舆论也认识到，用一刀切的方法抵制外资对办铁路无益，“反以致国民经济之压迫，不若借外债，资以开利源，而助国民经济之发达，巩固国家财政之基础之为优。”盛宣怀、梁启超也持此种看法。商办铁路公司一方面是被资金短绌问题所困扰，另一方面又存在经营不善、贪污挪用、亏损严重的问题。铁路建设领域各自为政、各行其是蔚然成风，商办铁路公司的铁路建设就成了无全盘规划的“圈地运动”。
面对铁路国有化的世界趋势，在商办铁路公司举步维艰的情况下，清政府试图将实行铁路国有化作为一种经济政策确定下来。1908年中央政府借规划全国铁路之名，下令切实勘查各省商办铁路的工程款项，已经流露出把筑路权收归国有的意图。20世纪初期的中国国内，不单是政府层，国内大舆论也开始呈现出实行铁路国有的趋势。曾经引领国人成功收回路权的张之洞此时也在修正自己的铁路建设融资理念。1909年6月，张之洞代表清政府与英、法、德三国银行团签订了借款合同草约，总计借款为550万英镑，利息5厘，以建造陷入困境中的粤汉与川汉铁路。在当时，这一草约的签定并没有引起政府所担心的过于猛烈的反对声浪。民间舆论也有进一步向政府倾斜的迹象。随着津浦铁路、沪杭甬铁路、湖广铁路的借款基本洽谈完成，拒款运动已发生明显转向，在这种隋况下，报章开始公开讨论铁路外债的可行性问题。1911年初，盛宣怀在度支部尚书载泽的支持下接任邮传部尚书一职后，更加快了国有政策的正式出台。
1911年5月9日，清政府正式发布“铁路国有”上谕（全文）。5月13日，长沙市面就出现保路揭帖，次日长沙发生万人游行请愿，18日，第一次衙门请愿爆发；在川汉线已动工的湖北，盛宣怀为强夺股权，下令停工，引发宜昌商人、民众聚集抗争，清廷调集军队弹压，双方发生冲突，死伤20多人，这是保路运动的第一次流血事件。6月16日，咨议局议长蒲殿俊、副议长罗纶在成都召集川汉股东临时大会，提出“文明保路”，聚集股东达2400名，次日，闻讯而来的各界学生、市民使与会人数达到4000以上。
1911年6月17日，四川民间各团体成立“四川保路同志会”，推举四川谘议局议长蒲殿俊为会长，副议长罗纶为副会长，张贴文告，四处讲演，甚至上京请愿。8月5日，在成都召开川汉铁路股东特别大会，8月24日，群众开展罢市罢课，9月1日，在川汉铁路公司股东会议的号召下，开展抗粮抗捐活动。9月7日，保路同志会代表率数万人陈请阻止川汉铁路钦差大臣端方进入四川，“赵督未允代表，众即言辞激烈，赵督当场喝令狙击”。手无缚鸡之力的蒲殿俊、罗纶等九人当场被捕，这便是死伤众多、骇人听闻的“成都血案”。

1911年6月中旬，四川保路运动升级，清政府意识到了事态的严重性，抽调包括湖北新军在内的多省力量前往四川弹压暴动风潮。这一事件对湖北影响很大，前往四川的湖北新军部队是革命党人占多数的新军三十二标，但是他们拒绝到四川，准备在进军荆州和宜昌的路上杀掉大臣端方。被居正阻止。

1911年9月8日，成都附近农民在中国同盟会和会党组织哥老会的领导下组成保路同志军起义，围攻省城，与清兵交战，附近州县群众纷纷响应，几天内队伍发展到20多万。荣县独立，荣县成为全中国第一个脱离清王朝的政权。清朝政府得知激起民变，成都被围后，非常惊恐，忙将四川总督赵尔丰免职，并调渝汉铁路督办端方署理四川总督，率一部分湖北新军入川。9月中旬，端方抽调了1/3湖北新军军力取道宜昌，进入四川，武昌登时空虚。

1911年9月14日，湖北共进会、文学社两个革命团体领导人孙武、刘复基、刘公、蒋翊武、王宪章、居正等人在雄楚楼十号刘公寓所集会，以为湖北新军主力调走后，武昌空虚，大有箭在弦上不得不发之
势。鉴于对形势的共识，刘复基建议如果此刻发动起义，共进会、文学社的名义都应该暂时搁置不用，大家都使用武昌革命党人的名义进行合作，“事成则卿，不成则烹”，同心协力，争取胜利。9月23日，湖北境内最有影响的两个革命组织共进会、文学社宣布合并，大家决定日后不再分彼此，一律统称“武昌革命党人”。第二天召开干部会议，组建了起义的领导机构——起义总指挥部。文学社负责人蒋翊武被推举为总指挥，共进会负责人孙武被推举为参谋长。起义时间定为10月6日（旧历八月十五，中秋节），后因准备不足，起义日期推迟到10月16日。9月24日，召集各部队代表一百多人举行会议。详细讨论军事行动后临时政府的框架及人选，确定各参战部队行动方案和联络方式。

"""
input2="""
第十三回 秦可卿死封龙禁尉 王熙凤协理宁国府

    话说凤姐儿自贾琏送黛玉往扬州去后，心中实在无趣，每到晚间，不过和平儿说笑一回，就胡乱睡了。

    这日夜间，正和平儿灯下拥炉倦绣，早命浓薰绣被，二人睡下，屈指算行程该到何处，不知不觉已交三鼓。平儿已睡熟了。凤姐方觉星眼微朦，恍惚只见秦氏从外走来，含笑说道：“婶子好睡！我今日回去，你也不送我一程。因娘儿们素日相好，我舍不得婶子，故来别你一别。还有一件心愿未了，非告诉婶子，别人未必中用。”

    凤姐听了，恍惚问道：“有何心愿？你只管托我就是了。”秦氏道：“婶婶，你是个脂粉队里的英雄，连那些束带顶冠的男子也不能过你，你如何连两句俗语也不晓得？常言。月满则亏，水满则溢&#039;，又道是。登高必跌重&#039;。如今我们家赫赫扬扬，已将百载，一日倘或乐极悲生，若应了那句。树倒猢狲散&#039;的俗语，岂不虚称了一世的诗书旧族了！”凤姐听了此话，心胸大快，十分敬畏，忙问道：这话虑的极是，但有何法可以永保无虞？”秦氏冷笑道：“婶子好痴也。否极泰来，荣辱自古周而复始，岂人力能可保常的。但如今能于荣时筹画下将来衰时的世业，亦可谓常保永全了。

    即如今日诸事都妥，只有两件未妥，若把此事如此一行，则后日可保永全了。

    凤姐便问何事。秦氏道：“目今祖茔虽四时祭祀，只是无一定的钱粮，第二，家塾虽立，无一定的供给。依我想来，如今盛时固不缺祭祀供给，但将来败落之时，此二项有何出处？莫若依我定见，趁今日富贵，将祖茔附近多置田庄房舍地亩，以备祭祀供给之费皆出自此处，将家塾亦设于此。合同族中长幼，大家定了则例，日后按房掌管这一年的地亩，钱粮，祭祀，供给之事。如此周流，又无争竞，亦不有典卖诸弊。便是有了罪，凡物可入官，这祭祀产业连官也不入的。便败落下来，子孙回家读书务农，也有个退步，祭祀又可永继。若目今以为荣华不绝，不思后日，终非长策。眼见不日又有一件非常喜事，真是烈火烹油，鲜花着锦之盛。要知道，也不过是瞬间的繁华，一时的欢乐，万不可忘了那。盛筵必散&#039;的俗语。此时若不早为后虑，临期只恐后悔无益了。”凤姐忙问：“有何喜事？”秦氏道：“天机不可泄漏。只是我与婶子好了一场，临别赠你两句话，须要记着。”因念道：

    三春过后诸芳尽，各自须寻各自门。凤姐还欲问时，只听二门上传事云板连叩四下，将凤姐惊醒。人回：“东府蓉大奶奶没了。”凤姐闻听，吓了一身冷汗，出了一回神，只得忙忙的穿衣，往王夫人处来。

    彼时合家皆知，无不纳罕，都有些疑心。那长一辈的想他素日孝顺，平一辈的想他素日和睦亲密，下一辈的想他素日慈爱，以及家中仆从老小想他素日怜贫惜贱，慈老爱幼之恩，莫不悲嚎痛哭者。

    闲言少叙，却说宝玉因近日林黛玉回去，剩得自己孤в，也不和人顽耍，每到晚间便索然睡了。如今从梦中听见说秦氏死了，连忙翻身爬起来，只觉心中似戳了一刀的不忍，哇的一声，直奔出一口血来。袭人等慌慌忙忙上来д扶，问是怎么样，又要回贾母来请大夫。宝玉笑道：“不用忙，不相干，这是急火攻心，血不归经。”说着便爬起来，要衣服换了，来见贾母，即时要过去。袭人见他如此，心中虽放不下，又不敢拦，只是由他罢了。贾母见他要去，因说：“才г气的人，那里不干净，二则夜里风大，等明早再去不迟。”宝玉那里肯依。贾母命人备车，多派跟随人役，拥护前来。一直到了宁国府前，只见府门洞开，两边灯笼照如白昼，乱烘烘人来人往，里面哭声摇山振岳。宝玉下了车，忙忙奔至停灵之室，痛哭一番。然后见过尤氏。谁知尤氏正犯了胃疼旧疾，睡在床上。然后又出来见贾珍。彼时贾代儒，代修，贾敕，贾效，贾敦，贾赦，贾政，贾琮，贾e，贾珩，贾e，贾琛，贾琼，贾ж，贾蔷，贾菖，贾菱，贾芸，贾芹，贾蓁，贾萍，贾藻，贾蘅，贾芬，贾芳，贾兰，贾菌，贾芝等都来了。贾珍哭的泪人一般，正和贾代儒等说道：“合家大小，远近亲友，谁不知我这媳妇比儿子还强十倍。如今伸腿去了，可见这长房内绝灭无人了。”说着又哭起来。众人忙劝：“人已辞世，哭也无益，且商议如何料理要紧。”贾珍拍手道：“如何料理，不过尽我所有罢了！”正说着，只见秦业，秦钟并尤氏的几个眷属尤氏姊妹也都来了。贾珍便命贾琼，贾琛，贾ж，贾蔷四个人去陪客，一面吩咐去请钦天监阴阳司来择日，择准停灵七七四十九日，三日后开丧送讣闻。这四十九日，单请一百单八众禅僧在大厅上拜大悲忏，超度前亡后化诸魂，以免亡者之罪，另设一坛于天香楼上，是九十九位全真道士，打四十九日解冤洗业醮。然后停灵于会芳园中，灵前另外五十众高僧，五十众高道，对坛按七作好事。那贾敬闻得长孙媳死了，因自为早晚就要飞升，如何肯又回家染了红尘，将前功尽弃呢，因此并不在意，只凭贾珍料理。

    贾珍见父亲不管，亦发恣意奢华。看板时，几副杉木板皆不中用。可巧薛蟠来吊问，因见贾珍寻好板，便说道：“我们木店里有一副板，叫作什么樯木，出在潢海铁网山上，作了棺材，万年不坏。这还是当年先父带来，原系义忠亲王老千岁要的，因他坏了事，就不曾拿去。现在还封在店内，也没有人出价敢买。你若要，就抬来使罢。”贾珍听说，喜之不尽，即命人抬来。大家看时，只见帮底皆厚八寸，纹若槟榔，味若檀麝，以手扣之，玎3如金玉。大家都奇异称赞。贾珍笑问：“价值几何？”薛蟠笑道：“拿一千两银子来，只怕也没处买去。什么价不价，赏他们几两工钱就是了。”贾珍听说，忙谢不尽，即命解锯糊漆。贾政因劝道：“此物恐非常人可享者，殓以上等杉木也就是了。”此时贾珍恨不能代秦氏之死，这话如何肯听。因忽又听得秦氏之丫鬟名唤瑞珠者，见秦氏死了，他也触柱而亡。此事可罕，合族人也都称叹。贾珍遂以孙女之礼敛殡，一并停灵于会芳园中之登仙阁。小丫鬟名宝珠者，因见秦氏身无所出，乃甘心愿为义女，誓任摔丧驾灵之任。

    贾珍喜之不尽，即时传下，从此皆呼宝珠为小姐。那宝珠按未嫁女之丧，在灵前哀哀欲绝。于是，合族人丁并家下诸人，都各遵旧制行事，自不得紊乱。

    贾珍因想着贾蓉不过是个黉门监，灵幡经榜上写时不好看，便是执事也不多，因此心下甚不自在。可巧这日正是首七第四日，早有大明宫掌宫内相戴权，先备了祭礼遣人来，次后坐了大轿，打伞鸣锣，亲来上祭。贾珍忙接着，让至逗蜂轩献茶。贾珍心中打算定了主意，因而趁便就说要与贾蓉捐个前程的话。戴权会意，因笑道：“想是为丧礼上风光些。”贾珍忙笑道：“老内相所见不差。”戴权道：“事倒凑巧，正有个美缺，如今三百员龙禁尉短了两员，昨儿襄阳侯的兄弟老三来求我，现拿了一千五百两银子，送到我家里。你知道，咱们都是老相与，不拘怎么样，看着他爷爷的分上，胡乱应了。还剩了一个缺，谁知永兴节度使冯胖子来求，要与他孩子捐，我就没工夫应他。既是咱们的孩子要捐，快写个履历来。”贾珍听说，忙吩咐：“快命书房里人恭敬写了大爷的履历来。”小厮不敢怠慢，去了一刻，便拿了一张红纸来与贾珍。贾珍看了，忙送与戴权。看时，上面写道：

    江南江宁府江宁县监生贾蓉，年二十岁。曾祖，原任京营节度使世袭一等神威将军贾代化，祖，乙卯科进士贾敬，父，世袭三品爵威烈将军贾珍。戴权看了，回手便递与一个贴身的小厮收了，说道：“回来送与户部堂官老赵，说我拜上他，起一张五品龙禁尉的票，再给个执照，就把这履历填上，明儿我来兑银子送去。”小厮答应了，戴权也就告辞了。贾珍十分款留不住，只得送出府门。临上轿，贾珍因问：“银子还是我到部兑，还是一并送入老内相府中？”戴权道：若到部里，你又吃亏了。不如平准一千二百两银子，送到我家就完了。贾珍感谢不尽，只说。待服满后，亲带小犬到府叩谢。于是作别。

    接着，便又听喝道之声，原来是忠靖侯史鼎的夫人来了。王夫人，邢夫人，凤姐等刚迎入上房，又见锦乡侯，川宁侯，寿山伯三家祭礼摆在灵前。少时，三人下轿，贾政等忙接上大厅。如此亲朋你来我去，也不能胜数。只这四十九日，宁国府街上一条白漫漫人来人往，花簇簇官去官来。

    贾珍命贾蓉次日换了吉服，领凭回来。灵前供用执事等物俱按五品职例。灵牌疏上皆写“天朝诰授贾门秦氏恭人之灵位”。会芳园临街大门洞开，旋在两边起了鼓乐厅，两班青衣按时奏乐，一对对执事摆的刀斩斧齐。更有两面朱红销金大字牌对竖在门外，上面大书：“防护内廷紫禁道御前侍卫龙禁尉”。对面高起着宣坛，僧道对坛榜文，榜上大书：“世袭宁国公冢孙妇，防护内廷御前侍卫龙禁尉贾门秦氏恭人之丧。四大部州至中之地，奉天承运太平之国，总理虚无寂静教门僧录司正堂万虚，总理元始三一教门道录司正堂叶生等，敬谨修斋，朝天叩佛”，以及“恭请诸伽蓝，揭谛，功曹等神，圣恩普锡，神威远镇，四十九日消灾洗业平安水陆道场”等语，亦不消烦记。

    只是贾珍虽然此时心意满足，但里面尤氏又犯了旧疾，不能料理事务，惟恐各诰命来往，亏了礼数，怕人笑话，因此心中不自在。当下正忧虑时，因宝玉在侧问道：“事事都算安贴了，大哥哥还愁什么？”贾珍见问，便将里面无人的话说了出来。宝玉听说笑道：“这有何难，我荐一个人与你权理这一个月的事，管必妥当。”贾珍忙问：“是谁？”宝玉见座间还有许多亲友，不便明言，走至贾珍耳边说了两句。贾珍听了喜不自禁，连忙起身笑道：“果然安贴，如今就去。”说着拉了宝玉，辞了众人，便往上房里来。

    可巧这日非正经日期，亲友来的少，里面不过几位近亲堂客，邢夫人，王夫人，凤姐并合族中的内眷陪坐。闻人报：“大爷进来了。”唬的众婆娘唿的一声，往后藏之不迭，独凤姐款款站了起来。贾珍此时也有些病症在身，二则过于悲痛了，因拄个拐踱了进来。邢夫人等因说道：“你身上不好，又连日事多，该歇歇才是，又进来做什么？”贾珍一面扶拐，扎挣着要蹲身跪下请安道乏。

    邢夫人等忙叫宝玉搀住，命人挪椅子来与他坐。贾珍断不肯坐，因勉强陪笑道：“侄儿进来有一件事要求二位婶子并大妹妹。”邢夫人等忙问：“什么事？”贾珍忙笑道：“婶子自然知道，如今孙子媳妇没了，侄儿媳妇偏又病倒，我看里头着实不成个体统。怎么屈尊大妹妹一个月，在这里料理料理，我就放心了。”邢夫人笑道：“原来为这个。你大妹妹现在你二婶子家，只和你二婶子说就是了。”王夫人忙道：“他一个小孩子家，何曾经过这样事，倘或料理不清，反叫人笑话，倒是再烦别人好。”贾珍笑道：“婶子的意思侄儿猜着了，是怕大妹妹劳苦了。若说料理不开，我包管必料理的开，便是错一点儿，别人看着还是不错的。从小儿大妹妹顽笑着就有杀伐决断，如今出了阁，又在那府里办事，越发历练老成了。我想了这几日，除了大妹妹再无人了。婶子不看侄儿，侄儿媳妇的分上，只看死了的分上罢！”说着滚下泪来。

    王夫人心中怕的是凤姐儿未经过丧事，怕他料理不清，惹人耻笑。今见贾珍苦苦的说到这步田地，心中已活了几分，却又眼看着凤姐出神。那凤姐素日最喜揽事办，好卖弄才干，虽然当家妥当，也因未办过婚丧大事，恐人还不伏，巴不得遇见这事。今见贾珍如此一来，他心中早已欢喜。先见王夫人不允，后见贾珍说的情真，王夫人有活动之意，便向王夫人道：“大哥哥说的这么恳切，太太就依了罢。”王夫人悄悄的道：“你可能么？”凤姐道：“有什么不能的。外面的大事已经大哥哥料理清了，不过是里头照管照管，便是我有不知道的，问问太太就是了。”王夫人见说的有理，便不作声。贾珍见凤姐允了，又陪笑道：“也管不得许多了，横竖要求大妹妹辛苦辛苦。

    我这里先与妹妹行礼，等事完了，我再到那府里去谢。“说着就作揖下去，凤姐儿还礼不迭。

    贾珍便忙向袖中取了宁国府对牌出来，命宝玉送与凤姐，又说：“妹妹爱怎样就怎样，要什么只管拿这个取去，也不必问我。只求别存心替我省钱，只要好看为上，二则也要同那府里一样待人才好，不要存心怕人抱怨。只这两件外，我再没不放心的了。”凤姐不敢就接牌，只看着王夫人。王夫人道：“你哥哥既这么说，你就照看照看罢了。只是别自作主意，有了事，打发人问你哥哥，嫂子要紧。”宝玉早向贾珍手里接过对牌来，强递与凤姐了。又问：“妹妹住在这里，还是天天来呢？若是天天来，越发辛苦了。不如我这里赶着收拾出一个院落来，妹妹住过这几日倒安稳。”凤姐笑道：“不用。那边也离不得我，倒是天天来的好。”贾珍听说，只得罢了。然后又说了一回闲话，方才出去。

    一时女眷散后，王夫人因问凤姐：“你今儿怎么样？”凤姐儿道：“太太只管请回去，我须得先理出一个头绪来，才回去得呢。”王夫人听说，便先同邢夫人等回去，不在话下。

    这里凤姐儿来至三间一所抱厦内坐了，因想：头一件是人口混杂，遗失东西，第二件，事无专执，临期推委，第三件，需用过费，滥支冒领，第四件，任无大小，苦乐不均，第五件，家人豪纵，有脸者不服钤束，无脸者不能上进。此五件实是宁国府中风俗，不知凤姐如何处治，且听下回分解。正是：

    金紫万千谁治国，裙钗一二可齐家。
"""
input3="""
低纬度气候：

　　低纬度的气候主要受赤道气团和热带气团所控制。影响气候的主要环流系统有赤道辐合带、瓦克环流、信风、赤道西风、热带气旋和副热带高压。全年地气系统的辐射差额是入超的，因此气温全年皆高，最冷月平均气温在15～18℃以上，全年水分可能蒸散量在130cm以上。本带可分为5个气候型，其中热带干旱与半干旱气候型又可划分为3个亚型。

赤道多雨气候

　　分布于赤道及其南、北5°～10°以内，宽窄不一，主要分布在非洲扎伊尔河流域、南美亚马孙河流域和亚洲与大洋洲间的从苏门答腊岛到伊里安岛一带。这里全年正午太阳高度都很大，因此长夏无冬，各月平均气温在25°28℃，年平均气温在26℃左右。气温年较差一般小于3℃，日较差可达6～12℃。由于全年皆在赤道气团控制下，风力微弱，以辐合上升气流为主，多雷阵雨，因此全年多雨，无干季，年降水量在2000mm以上，最少月在60mm以上。但降水量的年际变化很大，这与赤道辐合带位置的变动有关。

热带海洋性气候

　　分布在南北纬10°～25°信风带大陆东岸及热带海洋中的若干岛屿上。这里正当迎风海岸，全年盛行热带海洋气团，气候具有海洋性，最热月平均气温在28℃左右，最冷月平均气温在18～25℃之间，气温年较差、日较差皆小。由于东风（信风）带来湿热的海洋气团，所以除对流雨、热带气旋雨外，还多地形雨，降水量充沛。年降水量在1000mm以上，一般以5～10月较集中，无明显变化。

热带干湿季气候

　　大致分布在南北半球5°～25°之间。这里当正午太阳高度较小时，位于信风带下，受热带大陆气团控制，盛行下沉气流，是为干季。当正午太阳高度较大时，赤道辐合带移来，有潮湿的辐合上升气流，是为雨季。一年中至少有1～2个月为干季。湿季中蒸散量小于降水量。全年降水量在750～1600mm左右，降水变率很大。全年高温，最冷月平均气温在16～18℃以上，干季之末，雨季之前，气温最高，是为热季。

热带季风气候

　　分布在纬度10°到回归线附近的亚洲大陆东南部，如中国台湾南部、雷州半岛和海南岛，中南半岛，印度半岛大部，菲律宾，澳大利亚北部沿海等地。这里热带季风发达，一年中风向的季节变化明显。在热带大陆气团控制时，降水稀少。而当赤道气团控制时，降水丰沛，又有大量的热带气旋雨，年降水量多，一般在1500～2000mm，集中在 6～10月（北半球）。全年高温，年平均气温在20℃以上，年较差在3～10℃左右，春秋极短。

热带干旱与半干旱气候

　　分布在副热带及信风带的大陆中心和大陆西岸。在南、北半球各约以回归线为中心向南北伸展，平均位置大致在纬度15°～25°间。因干旱程度和气候特征不同，可分为热带干旱气候（5a）、热带（西岸）多雾干旱气候（5b）和热带半干旱气候（5c）三个亚型。5a，5c是热带大陆气团的源地，气温年较差、日较差都大，有极端最高气温。5a终年受副热带高压下沉气流控制，因此降水量极少。5c位于5a的外缘，大半年时间受副热带高压控制而干燥少雨，在太阳高度大的季节，赤道低压槽移来，有对流雨，因此出现一短暂的雨季。5b位于热带大陆西岸，有冷洋流经过，终年受海洋副热带高压下沉气流影响，多雾而少雨，降水量极小，但气温较凉，气温年较差、日较差皆小。

中纬度气候：
　　这里是热带气团和极地气团相互角逐的地带。影响气候的主要环流系统有极锋、盛行西风、温带气旋和反气旋、副热带高压和热带气旋等。该地带一年中辐射能收支差额的变化比较大，因此四季分明，最冷月的平均气温在15～18℃以下，有4～12个月平均气温在10℃以上。全年可能蒸散量在130～52.5cm之间。天气的非周期性变化和降水的季节变化都很显著。再加上北半球中纬度地带大陆面积较大，受海陆的热力对比和高耸庞大地形的影响，使得本带气候更加错综复杂。本带共分8个气候型。

副热带干旱与半干旱气候

　　分布在热带干旱气候向高纬度的一侧，约在南北纬25°～35°的大陆西岸和内陆地区。它是在副热带高压下沉气流和信风带背岸风的作用下形成的。因干旱程度不同可分为干旱6a与半干旱6b两亚型。 6a副热带干旱气候具有少云、少雨、日照强和夏季气温特高等特征。但凉季气温比5a型低，气温年较差较5a型大，达20℃以上。凉季有少量气旋雨，土壤蓄水量略大于5a型。6b副热带半干旱气候位于6a区外缘。夏季气温比6a型低，冬季降水量比6a型稍多。

副热带季风气候

　　分布于副热带亚欧大陆东岸，约以30°N为中心，向南北各伸展5°左右。这里是热带海洋气团与极地大陆气团交绥角逐的地带，夏秋季节又受热带气旋活动的影响，因此夏热湿、冬温干，最热月平均气温在22℃以上，最冷月平均气温在0～15℃左右，气温年较差约在15～25℃左右。降水量在750～1000mm以上。夏雨较集中，无明显干季。四季分明，无霜期长。 澳大利亚气候分类图

副热带湿润气候

　　分布于南北美洲、非洲和澳大利亚大陆副热带东岸，约为南北纬20°～35°。冬季受极地大陆气团影响，夏季受海洋高压西缘流来的潮湿海洋气团的控制。由于所处大陆面积小，未形成季风气候。冬夏温差比季风区小，降水的季节分配比季风区均匀。

副热带夏干气候（地中海气候）

　　分布于副热带大陆西岸30°～40°之间的地带。这里受副热带高压季节移动的影响，在夏季正位于副高中心范围之内或在其东缘，气流是下沉的，因此干燥少雨，日照强烈。冬季副高移向较低纬度，这里受西风带控制，锋面、气旋活动频繁，带来大量降水。全年降水量在300～1000mm左右。冬季气温比较暖和，最冷月平均气温在4～10℃左右。因夏温不同，分为两个亚型。9a凉夏型，贴近冷洋流海岸，夏季凉爽多雾，少雨，最热月平均气温在22℃以下，最冷月平均气温在10℃以上。9b暖夏型，离海岸较远，夏季干热，最热月平均气温在22℃以上，冬季温和湿润，气温年较差稍大。

温带海洋性气候

　　分布在温带大陆西岸约40°～60°的地带。这里终年盛行西风，受温带海洋气团控制，沿岸有暖洋流经过。冬暖夏凉，最冷月平均气温在0℃以上，最热月平均气温在22℃以下，气温年较差小，约在6～14℃左右。全年湿润有雨，冬季较多。年降水量750～1000mm左右，迎风山地可达2000mm以上。

温带季风气候

　　分布在亚欧大陆东岸约35°～55°的地带。这里冬季盛行偏北风，寒冷干燥，最冷月平均气温在0℃以下，南北气温差别大。夏季盛行东南风，温暖湿润，最热月平均气温在20℃以上，南北温差小。气温年较差比较大，全年降水量集中于夏季，降水分布由南向北，由沿海向内陆减少。天气的非周期性变化显著，冬季寒潮爆发时，气温在24小时内可下降10余摄氏度甚至20余摄氏度。

温带大陆性湿润气候

　　分布在亚欧大陆温带海洋性气候区的东侧，北美100°W以东的温带的地区。冬季受极地大陆气团控制而寒冷，有少量气旋性降水。夏季受热带海洋气团的侵入，降水量较多，但不像季风区那样高度集中。这里季节鲜明，天气变化剧烈。

温带干旱与半干旱气候

　　分布在35°～50°N的亚洲和北美洲大陆中心部分。由于距离海洋较远或受山地屏障，受不到海洋气团的影响，终年都在大陆气团的控制下，因此气候干燥，夏热冬寒，气温年较差很大。因干旱程度不同可分为温带干旱气候（13a）和温带半干旱气候（13b）两个亚型。

高纬度气候：
　　高纬度气候带盛行极地气团和冰洋气团。冰洋锋上有气旋活动。这里地气系统的辐射差额为负值，所以气温低，无真正的夏季。空气中水汽含量少，降水量小，但蒸发弱，年可能蒸散量小于52.5cm。本带可分为三个气候型。

副极地大陆性气侯

　　分布在50°N或55°N到65°N的地区。这里年可能蒸散量在35cm到52.5cm之间。冬季长，一年中至少有9个月为冬季。冬季黑夜时间长，正午太阳高度小，在欧亚大陆中部和偏东地区又为冷高压中心，风小、云少，地面辐射冷却剧烈，大陆性最强，冬温极低。夏季白昼时间长，7月平均气温在15℃以上，气温年较差特大。全年降水量甚少，集中于暖季降落，冬雪较少，但蒸发弱，融化慢，每年有5～7个月的积雪覆盖，积雪厚度在600～700mm左右，土壤冻结现象严重。由于暖季温度适中，又有一定降水量，适宜针叶林生长。

极地苔原气候

　　分布在北美洲和欧亚大陆的北部边缘、格陵兰沿海的一部分和北冰洋中的若干岛屿中。在南半球则分布在马尔维纳斯群岛（福克兰群岛）、南设得兰群岛和南奥克尼群岛等地。年可能蒸散量小于35cm。全年皆冬，一年中只有1～4个月月平均气温在0℃～10℃左右。其纬度位置已接近或位于极圈以内，所以极昼、极夜现象已很明显。在极夜期间气温很低，但邻近海洋比副极地大陆性气候稍高。最冷月平均气温在-20℃～40℃之间。最热月平均气温在1～5℃左右。在7，8月份，夜间气温仍可降到0℃以下。在冰洋锋上有一定降水，一般年降水量在200～300mm左右。在内陆地区尚不足200 mm，大都为干雪，暖季为雨或湿雪。由于风速大，常形成雪雾，能见度不佳，地面积雪面积不大。自然植被只有苔藓、地衣及小灌木等，构成了苔原景观。

极地冰原气候

　　分布在格陵兰、南极大陆和北冰洋的若干岛屿上。这里是冰洋气团和南极气团的源地，全年严寒，各月平均气温皆在0℃以下，具有全球的最低年平均气温。一年中有长时期的极昼、极夜现象。全年降水量小于250 mm，皆为干雪，不会融化，长期累积形成很厚的冰原。长年大风，寒风夹雪，能见度恶劣。

高地气候：
在高地地带随着高度的增加，气候诸要素也随着发生变化，导致高山气候具有明显的垂直地带性。为了区分因高度影响和因纬度等因素影响的气候，也因为高山气候仅限于局部范围，所以高地气候单列为一大类而没有包括在低地分类系统内。 高山气候具有明显的垂直地带性，这种垂直地带性又因高山所在地的纬度和区域气候条件而有所不同，其特征如下：

（1）山地垂直气候带的分异因所在地的纬度和山地本身的高差而异在低纬山地，山麓为赤道或热带气候，随着海拔的增加，地表热量和水分条件逐渐变化，垂直气候带依次发生。这种变化类似于低地随纬度的增加而发生的变化。如果山地的纬度较高，气候垂直带的分异就减少。如果山地的高差较小，气候垂直带的分异也就较小。 

（2）山地垂直气候带具有所在地大气候类型的“烙印”例如，赤道山地从山麓到山顶都具有全年季节变化不明显的特征。珠穆朗玛峰和长白山都具有季风气候特色。

（3）湿润气候区山地垂直气候的分异主要以热量条件为垂直差异的决定因素 而干旱、半干旱气候区，山地垂直气候的分异，与热量和湿润状况都有密切关系。这种地区的干燥度都是山麓大，随着海拔的增高，干燥度逐渐减小。

（4）同一山地还因坡向、坡度及地形起伏、凹凸、显隐等局地条件不同，气候的垂直变化各不相同山坡暖带、山谷冷湖即为一例。山地气候确有“十里不同天”之变。

（5）山地的垂直气候带与随纬度而异的水平气候带在成因和特征上都有所不同。
"""
input4="""
根据其细胞的基本结构，可以将所有活生物体分为两组之一：原核生物和真核生物。原核生物是由缺乏细胞核或任何膜包裹的细胞器的细胞组成的生物。真核生物是由具有膜结合核的细胞组成的生物，该核保留遗传物质以及膜结合的细胞器。

了解细胞和细胞膜

该电池是我们现代的生活和居住的东西定义的一个基本组成部分。细胞被认为是生命的基本组成部分，并用于难以理解的“活”的定义。

细胞使化学过程保持整齐和分隔，因此单个细胞过程不会干扰其他细胞，并且细胞可以进行代谢，繁殖等业务。为此，细胞成分被包裹在膜中，该膜可作为外部屏障世界和细胞的内部化学。的细胞膜为选择性屏障，这意味着它可以让某些化学品和其他人进行。这样做可以维持细胞存活所需的化学平衡。

细胞膜通过三种方式调节化学物质在细胞内外的穿越：

扩散（溶质分子使浓度最小化并因此从较高浓度的区域移向较低浓度的区域直到浓度相等的趋势）
渗透（溶剂跨过选择性边界移动，以平衡无法跨边界移动的溶质的浓度）
选择性运输（通过膜通道和膜泵）
原核生物:

原核生物是由缺乏细胞核或任何膜包裹的细胞器的细胞组成的生物。这意味着原核生物中的遗传物质DNA未结合在核内。此外，与原核生物相比，DNA在原核生物中的结构更少：在原核生物中，DNA是一个单环，而在真核生物中，DNA被组织成染色体。大多数原核生物仅由单个细胞（单细胞）组成，但也有少数由细胞集合（多细胞）组成。

科学家将原核生物分为细菌和古细菌两大类。食品中发现了一些细菌，包括大肠杆菌，沙门氏菌和李斯特菌，它们可能导致疾病。1 其他实际上有助于人体消化和其他功能。2 古细菌被发现是一种独特的生命形式，能够在热液喷口或北极冰等极端环境中无限期地生活。

典型的原核细胞可能包含以下部分：

细胞壁：围绕并保护细胞的膜
细胞质：细胞内除核外的所有物质
鞭毛和菌毛：在某些原核细胞的外部发现基于蛋白质的细丝
核样：细胞中保留遗传物质的核样区域
质粒：可以独立繁殖的小分子DNA
真核生物:

真核生物是由具有膜结合核（以染色体形式保存DNA）以及膜结合细胞器的细胞组成的生物。真核生物可以是多细胞或单细胞生物。所有的动物都是真核生物。其他真核生物包括植物，真菌和原生生物。

典型的真核细胞被质膜包围，并包含许多具有各种功能的不同结构和细胞器。例子包括染色体（以基因形式携带遗传信息的核酸和蛋白质的结构）和线粒体（通常被称为“细胞的强大分子”


"""

## Bubble Map

In [11]:
%%time
instruction='我将给你一个文本，请找出文本的中心事件，并根据文本总结中心事件的相关属性和简短地描述，并按下面格式输出。\n \
## 中心事件:[中心事件名称]\n \
### 属性1:[属性名]\n \
#### 属性描述:[简短的描述]\n\
### 属性2:[属性名]\n \
#### 属性描述:[简短的描述]\n\
### 属性n:[属性名]\n \
#### 属性描述:[简短的描述]\n\
<|im_end|>\
'
input=input1
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        input,
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)
#outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
respond=tokenizer.batch_decode(outputs)[0].split('### 回答:\n')[-1]
save_llm_respond(respond,'LLM_respond_for_Bubble_Map.txt')

下面是描述任务的指令，并配有提供进一步上下文的输入。请按指令要求给出合适的回答。

### 指令:
我将给你一个文本，请找出文本的中心事件，并根据文本总结中心事件的相关属性和简短地描述，并按下面格式输出。
 ## 中心事件:[中心事件名称]
 ### 属性1:[属性名]
 #### 属性描述:[简短的描述]
### 属性2:[属性名]
 #### 属性描述:[简短的描述]
### 属性n:[属性名]
 #### 属性描述:[简短的描述]
<|im_end|>

### 输入:

兴建铁路作为一种官方政策是在新政之前就确立的，对近代中国的铁路建设起到了积极作用。自甲午战争结束到1911年，清王朝共建铁路4936公里，占近代中国所修铁路将近40%，而其中绝大部分是在新政时期建设而成。但主要依靠外资资助修筑的就长达4029公里，约占总长度的81%。但是作为回报，清政府不得不把铁路的管理权、用人权、稽核权、购料权拱手让给外国的借款公司，这就导致了中国利权的严重流失。即便如此，李鸿章在世时还是不无遗憾地说道：“目下经费难筹，必借洋款。”

1903年12月2日，满清政府颁布了《铁路简明章程》，规定“无论华人、洋人”均可向“督抚衙门递呈请办铁路”，而且包括“干路或枝路”，华人投资50万两以上实有成效者，朝廷还要“专折请旨给予优奖”。各省绅商随之掀起规模巨大的收回路权运动，这种行动在朝野上下产生了更大的共鸣。粤汉铁路于1900年承包给美国永兴公司建造，湖南、湖北和广东三省绅士以美国商人违反合同为由，要求收回粤汉铁路筑路权，转而自己修筑，此举得到张之洞的鼎力支持，湖南士绅王先谦从中周旋，终于在1905年8月以赔款675万美元为代价，赎回原先由美国公司发行的股票，废除了与美商签订的建设粤汉铁路的合同。在这个成功案例的鼓舞下，浙江、江苏、山东、直隶、吉林、云南等省收回路权的工作也先后付诸实施，要求赎回沪宁铁路、苏甬杭铁路、广九铁路承办权的社会呼声越来越高，并实际收回了一批筑路权，铁路商办运动进入高潮时期。在此后4年间，全国在15个省创设了18个大型铁路公司，其中17个是商办、官督商办或官商合办的。

清末商办铁路虽然资本匮乏，但是在国人爱国主义的热情鼓舞之下，均设置了严格的受款规定，坚决拒绝外资、不招洋股、不借洋债，故而必须进行大量的民间融资。铁路建设费用极大，一时间又筹不到足够资金，

## Double Bubble Map

In [14]:
%%time
instruction='我将给你一个用于比较两个事物的文本，请找出两个事物，并根据下面格式总结两个事物的共同点和不同点。\n \
## 比较对象: [对象1]，[对象2]\n \
### 相同点:\n \
#### 相同点1:[简短的描述]\n\
#### 相同点2:\n \
#### 相同点n:\n \
### 对象1独有的特点:\n\
#### 特点1:[简短的描述]\n \
#### 特点2:\n \
#### 特点n:\n \
### 对象2独有的特点:\n\
#### 特点1:[简短的描述]\n \
#### 特点2:\n \
#### 特点n:\n \
<|im_end|>\
'
input=input4
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        input,
        "", # output 
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)
respond=tokenizer.batch_decode(outputs)[0].split('### 回答:\n')[-1]
save_llm_respond(respond,'LLM_respond_for_Double_Bubble_Map.txt')

下面是描述任务的指令，并配有提供进一步上下文的输入。请按指令要求给出合适的回答。

### 指令:
我将给你一个用于比较两个事物的文本，请找出两个事物，并根据下面格式总结两个事物的共同点和不同点。
 ## 比较对象: [对象1]，[对象2]
 ### 相同点:
 #### 相同点1:简短的描述
#### 相同点2:
 #### 相同点n:
 ### 对象1独有的特点:
#### 特点1:简短的描述
 #### 特点2:
 #### 特点n:
 ### 对象2独有的特点:
#### 特点1:简短的描述
 #### 特点2:
 #### 特点n:
 <|im_end|>

### 输入:

根据其细胞的基本结构，可以将所有活生物体分为两组之一：原核生物和真核生物。原核生物是由缺乏细胞核或任何膜包裹的细胞器的细胞组成的生物。真核生物是由具有膜结合核的细胞组成的生物，该核保留遗传物质以及膜结合的细胞器。

了解细胞和细胞膜

该电池是我们现代的生活和居住的东西定义的一个基本组成部分。细胞被认为是生命的基本组成部分，并用于难以理解的“活”的定义。

细胞使化学过程保持整齐和分隔，因此单个细胞过程不会干扰其他细胞，并且细胞可以进行代谢，繁殖等业务。为此，细胞成分被包裹在膜中，该膜可作为外部屏障世界和细胞的内部化学。的细胞膜为选择性屏障，这意味着它可以让某些化学品和其他人进行。这样做可以维持细胞存活所需的化学平衡。

细胞膜通过三种方式调节化学物质在细胞内外的穿越：

扩散（溶质分子使浓度最小化并因此从较高浓度的区域移向较低浓度的区域直到浓度相等的趋势）
渗透（溶剂跨过选择性边界移动，以平衡无法跨边界移动的溶质的浓度）
选择性运输（通过膜通道和膜泵）
原核生物:

原核生物是由缺乏细胞核或任何膜包裹的细胞器的细胞组成的生物。这意味着原核生物中的遗传物质DNA未结合在核内。此外，与原核生物相比，DNA在原核生物中的结构更少：在原核生物中，DNA是一个单环，而在真核生物中，DNA被组织成染色体。大多数原核生物仅由单个细胞（单细胞）组成，但也有少数由细胞集合（多细胞）组成。

科学家将原核生物分为细菌和古细菌两大类。食品中发现了一些细菌，包括大肠杆菌，沙门氏菌和李斯特菌，它们可能导致疾病。1 其他实际上有助于人体消化和其他功能。2 古细菌被发现是一种独特的生命形式，能够在热液喷口或北极

## Tree Map

In [39]:
%%time
instruction='我将给你一个用于介绍某个事物相关类别的文本，请找出中心事物，并根据下面格式简短地总结事物的主要类别。\n \
## 一级类别: [中心事物]\n \n \
### 二级类别1:\n \
- 三级类别1:[简短的描述]\n \
- 三级类别2:\n \
- 三级类别n:\n \
\n \
### 二级类别2:\n \
- 三级类别1:[简短的描述]\n \
- 三级类别2:\n \
- 三级类别n:\n \
\n \
### 二级类别m:\n \
- 三级类别1:[简短的描述]\n \
- 三级类别2:\n \
- 三级类别n:\n \
<|im_end|>\
'
input=input3

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        input,
        "", # output 
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)
respond=tokenizer.batch_decode(outputs)[0].split('### 回答:\n')[-1]
save_llm_respond(respond,'LLM_respond_for_tree_Map.txt')

下面是描述任务的指令，并配有提供进一步上下文的输入。请按指令要求给出合适的回答。

### 指令:
我将给你一个用于介绍某个事物相关类别的文本，请找出中心事物，并根据下面格式简短地总结事物的主要类别。
 ## 一级类别: [中心事物]
 
 ### 二级类别1:
 - 三级类别1:[简短的描述]
 - 三级类别2:
 - 三级类别n:
 
 ### 二级类别2:
 - 三级类别1:[简短的描述]
 - 三级类别2:
 - 三级类别n:
 
 ### 二级类别m:
 - 三级类别1:[简短的描述]
 - 三级类别2:
 - 三级类别n:
 <|im_end|>

### 输入:

低纬度气候：

　　低纬度的气候主要受赤道气团和热带气团所控制。影响气候的主要环流系统有赤道辐合带、瓦克环流、信风、赤道西风、热带气旋和副热带高压。全年地气系统的辐射差额是入超的，因此气温全年皆高，最冷月平均气温在15～18℃以上，全年水分可能蒸散量在130cm以上。本带可分为5个气候型，其中热带干旱与半干旱气候型又可划分为3个亚型。

赤道多雨气候

　　分布于赤道及其南、北5°～10°以内，宽窄不一，主要分布在非洲扎伊尔河流域、南美亚马孙河流域和亚洲与大洋洲间的从苏门答腊岛到伊里安岛一带。这里全年正午太阳高度都很大，因此长夏无冬，各月平均气温在25°28℃，年平均气温在26℃左右。气温年较差一般小于3℃，日较差可达6～12℃。由于全年皆在赤道气团控制下，风力微弱，以辐合上升气流为主，多雷阵雨，因此全年多雨，无干季，年降水量在2000mm以上，最少月在60mm以上。但降水量的年际变化很大，这与赤道辐合带位置的变动有关。

热带海洋性气候

　　分布在南北纬10°～25°信风带大陆东岸及热带海洋中的若干岛屿上。这里正当迎风海岸，全年盛行热带海洋气团，气候具有海洋性，最热月平均气温在28℃左右，最冷月平均气温在18～25℃之间，气温年较差、日较差皆小。由于东风（信风）带来湿热的海洋气团，所以除对流雨、热带气旋雨外，还多地形雨，降水量充沛。年降水量在1000mm以上，一般以5～10月较集中，无明显变化。

热带干湿季气候

　　大致分布在南北半球5°～25°之间。这里当正午太阳高度较小时，位于信风带下，受热带大陆气团控制，盛行下沉气流，是为干季。当正午太阳高度较大时，赤道辐合带移来，有潮湿的辐合

## Brace Map 

In [42]:
%%time
instruction='我将给你一个文本，请先找出中心事件，然后分背景、过程、意义三个部分总结文本，请按下面格式输出。\n \
## 中心事件:[中心事件名称]\
### 部分1：[名称]\n\
 - \
 - \
### 部分2：\n\
 - \
 - \
### 部分n：\n\
 - \
 - \
<|im_end|>\
'
input=input1
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        input,
        "", # output
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)
respond=tokenizer.batch_decode(outputs)[0].split('### 回答:\n')[-1]
save_llm_respond(respond,'LLM_respond_for_brace_Map.txt')

下面是描述任务的指令，并配有提供进一步上下文的输入。请按指令要求给出合适的回答。

### 指令:
我将给你一个文本，请先找出中心事件，然后分背景、过程、意义三个部分总结文本，请按下面格式输出。
 ## 中心事件:[中心事件名称]### 部分1：[名称]
 -  - ### 部分2：
 -  - ### 部分n：
 -  - <|im_end|>

### 输入:

兴建铁路作为一种官方政策是在新政之前就确立的，对近代中国的铁路建设起到了积极作用。自甲午战争结束到1911年，清王朝共建铁路4936公里，占近代中国所修铁路将近40%，而其中绝大部分是在新政时期建设而成。但主要依靠外资资助修筑的就长达4029公里，约占总长度的81%。但是作为回报，清政府不得不把铁路的管理权、用人权、稽核权、购料权拱手让给外国的借款公司，这就导致了中国利权的严重流失。即便如此，李鸿章在世时还是不无遗憾地说道：“目下经费难筹，必借洋款。”

1903年12月2日，满清政府颁布了《铁路简明章程》，规定“无论华人、洋人”均可向“督抚衙门递呈请办铁路”，而且包括“干路或枝路”，华人投资50万两以上实有成效者，朝廷还要“专折请旨给予优奖”。各省绅商随之掀起规模巨大的收回路权运动，这种行动在朝野上下产生了更大的共鸣。粤汉铁路于1900年承包给美国永兴公司建造，湖南、湖北和广东三省绅士以美国商人违反合同为由，要求收回粤汉铁路筑路权，转而自己修筑，此举得到张之洞的鼎力支持，湖南士绅王先谦从中周旋，终于在1905年8月以赔款675万美元为代价，赎回原先由美国公司发行的股票，废除了与美商签订的建设粤汉铁路的合同。在这个成功案例的鼓舞下，浙江、江苏、山东、直隶、吉林、云南等省收回路权的工作也先后付诸实施，要求赎回沪宁铁路、苏甬杭铁路、广九铁路承办权的社会呼声越来越高，并实际收回了一批筑路权，铁路商办运动进入高潮时期。在此后4年间，全国在15个省创设了18个大型铁路公司，其中17个是商办、官督商办或官商合办的。

清末商办铁路虽然资本匮乏，但是在国人爱国主义的热情鼓舞之下，均设置了严格的受款规定，坚决拒绝外资、不招洋股、不借洋债，故而必须进行大量的民间融资。铁路建设费用极大，一时间又筹不到足够资金，于是政府不得不出面协助，一时“租股、派股”盛极全国各地。湖北咨议局全体会议通过决议，后来又进一步提出拒款方案十项，

## Flow Map

In [17]:
%%time
instruction='我将给你一个文本，请按时间顺序将事件划分为若干个阶段(不超过5个),每个阶段包括若干个子事件，并按下面格式进行总结。\n \
## 阶段1:\n \
- 子事件1:[简短的描述]\n \
- 子事件2:\n \
- 子事件m:\n \
## 阶段2:\n \
- 子事件1:[简短的描述]\n \
- 子事件2:\n \
- 子事件m:\n \
## 阶段n:\n \
- 子事件1:[简短的描述]\n \
- 子事件2:\n \
- 子事件m:\n \
<|im_end|>\
'
input=input2
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        input,
        "", # output 
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)
respond=tokenizer.batch_decode(outputs)[0].split('### 回答:\n')[-1]
save_llm_respond(respond,'LLM_respond_for_flow_Map.txt')

下面是描述任务的指令，并配有提供进一步上下文的输入。请按指令要求给出合适的回答。

### 指令:
我将给你一个文本，请按时间顺序将事件划分为若干个阶段(不超过5个),每个阶段包括若干个子事件，并按下面格式进行总结。
 ## 阶段1:
 - 子事件1:[简短的描述]
 - 子事件2:
 - 子事件m:
 ## 阶段2:
 - 子事件1:[简短的描述]
 - 子事件2:
 - 子事件m:
 ## 阶段n:
 - 子事件1:[简短的描述]
 - 子事件2:
 - 子事件m:
 <|im_end|>

### 输入:

第十三回 秦可卿死封龙禁尉 王熙凤协理宁国府

    话说凤姐儿自贾琏送黛玉往扬州去后，心中实在无趣，每到晚间，不过和平儿说笑一回，就胡乱睡了。

    这日夜间，正和平儿灯下拥炉倦绣，早命浓薰绣被，二人睡下，屈指算行程该到何处，不知不觉已交三鼓。平儿已睡熟了。凤姐方觉星眼微朦，恍惚只见秦氏从外走来，含笑说道：“婶子好睡！我今日回去，你也不送我一程。因娘儿们素日相好，我舍不得婶子，故来别你一别。还有一件心愿未了，非告诉婶子，别人未必中用。”

    凤姐听了，恍惚问道：“有何心愿？你只管托我就是了。”秦氏道：“婶婶，你是个脂粉队里的英雄，连那些束带顶冠的男子也不能过你，你如何连两句俗语也不晓得？常言。月满则亏，水满则溢&#039;，又道是。登高必跌重&#039;。如今我们家赫赫扬扬，已将百载，一日倘或乐极悲生，若应了那句。树倒猢狲散&#039;的俗语，岂不虚称了一世的诗书旧族了！”凤姐听了此话，心胸大快，十分敬畏，忙问道：这话虑的极是，但有何法可以永保无虞？”秦氏冷笑道：“婶子好痴也。否极泰来，荣辱自古周而复始，岂人力能可保常的。但如今能于荣时筹画下将来衰时的世业，亦可谓常保永全了。

    即如今日诸事都妥，只有两件未妥，若把此事如此一行，则后日可保永全了。

    凤姐便问何事。秦氏道：“目今祖茔虽四时祭祀，只是无一定的钱粮，第二，家塾虽立，无一定的供给。依我想来，如今盛时固不缺祭祀供给，但将来败落之时，此二项有何出处？莫若依我定见，趁今日富贵，将祖茔附近多置田庄房舍地亩，以备祭祀供给之费皆出自此处，将家塾亦设于此。合同族中长幼，大家定了则例，日后按房掌管这一年的地亩，钱粮，祭祀，供给之事。如此周流，又无争竞，亦不有典卖诸弊。便是有了罪，凡物

## Multi-Flow Map 

In [27]:
%%time
instruction='我将给你一个描述了一个事件的文本，请找出事件名称，并简短地总结事件的原因和影响，按下面格式输出。\n \
## 事件:[事件名称]\n \
### 原因：\
- 原因1 \
- 原因2 \
- 原因n \
### 影响：\
- 影响1 \
- 影响2 \
- 影响m \
<|im_end|>\
'
input=input1
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        input,
        "", # output 
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)
respond=tokenizer.batch_decode(outputs)[0].split('### 回答:\n')[-1]
save_llm_respond(respond,'LLM_respond_for_Multi_flow_Map.txt')

下面是描述任务的指令，并配有提供进一步上下文的输入。请按指令要求给出合适的回答。

### 指令:
我将给你一个描述了一个事件的文本，请找出事件名称，并简短地总结事件的原因和影响，按下面格式输出。
 ## 事件:[事件名称]
 ### 原因：- 原因1 - 原因2 - 原因n ### 影响：- 影响1 - 影响2 - 影响m <|im_end|>

### 输入:

兴建铁路作为一种官方政策是在新政之前就确立的，对近代中国的铁路建设起到了积极作用。自甲午战争结束到1911年，清王朝共建铁路4936公里，占近代中国所修铁路将近40%，而其中绝大部分是在新政时期建设而成。但主要依靠外资资助修筑的就长达4029公里，约占总长度的81%。但是作为回报，清政府不得不把铁路的管理权、用人权、稽核权、购料权拱手让给外国的借款公司，这就导致了中国利权的严重流失。即便如此，李鸿章在世时还是不无遗憾地说道：“目下经费难筹，必借洋款。”

1903年12月2日，满清政府颁布了《铁路简明章程》，规定“无论华人、洋人”均可向“督抚衙门递呈请办铁路”，而且包括“干路或枝路”，华人投资50万两以上实有成效者，朝廷还要“专折请旨给予优奖”。各省绅商随之掀起规模巨大的收回路权运动，这种行动在朝野上下产生了更大的共鸣。粤汉铁路于1900年承包给美国永兴公司建造，湖南、湖北和广东三省绅士以美国商人违反合同为由，要求收回粤汉铁路筑路权，转而自己修筑，此举得到张之洞的鼎力支持，湖南士绅王先谦从中周旋，终于在1905年8月以赔款675万美元为代价，赎回原先由美国公司发行的股票，废除了与美商签订的建设粤汉铁路的合同。在这个成功案例的鼓舞下，浙江、江苏、山东、直隶、吉林、云南等省收回路权的工作也先后付诸实施，要求赎回沪宁铁路、苏甬杭铁路、广九铁路承办权的社会呼声越来越高，并实际收回了一批筑路权，铁路商办运动进入高潮时期。在此后4年间，全国在15个省创设了18个大型铁路公司，其中17个是商办、官督商办或官商合办的。

清末商办铁路虽然资本匮乏，但是在国人爱国主义的热情鼓舞之下，均设置了严格的受款规定，坚决拒绝外资、不招洋股、不借洋债，故而必须进行大量的民间融资。铁路建设费用极大，一时间又筹不到足够资金，于是政府不得不出面协助，一时“租股、派股”盛极全国各地。湖北咨议局全体会议通过决议，后来又进一步提出拒款方案十项，

## 整合封装

In [30]:
## 各种图的提示词
bubble_map_instruction='我将给你一个文本，请找出文本的中心事件，并根据文本总结中心事件的相关属性和简短的描述，并按下面格式输出。\n \
## 中心事件:[中心事件名称]\n \
### 属性1:[属性名]\n \
#### 属性描述:[简短的描述]\n\
### 属性2:[属性名]\n \
#### 属性描述:[简短的描述]\n\
### 属性n:[属性名]\n \
#### 属性描述:[简短的描述]\n\
<|im_end|>\
'
double_bubble_map_instruction='我将给你一个用于比较两个事物的文本，请找出两个事物，并根据下面格式总结两个事物的共同点和不同点。\n \
## 比较对象: [对象1]，[对象2]\n \
### 相同点:\n \
#### 相同点1:[简短的描述]\n\
#### 相同点2:\n \
#### 相同点n:\n \
### 对象1独有的特点:\n\
#### 特点1:[简短的描述]\n \
#### 特点2:\n \
#### 特点n:\n \
### 对象2独有的特点:\n\
#### 特点1:[简短的描述]\n \
#### 特点2:\n \
#### 特点n:\n \
<|im_end|>\
'
tree_map_instruction='我将给你一个用于介绍某个事物相关类别的文本，请找出中心事物，并根据下面格式简短地总结事物的主要类别。\n \
## 一级类别: [中心事物]\n \n \
### 二级类别1:\n \
- 三级类别1:[简短的描述]\n \
- 三级类别2:\n \
- 三级类别n:\n \
\n \
### 二级类别2:\n \
- 三级类别1:[简短的描述]\n \
- 三级类别2:\n \
- 三级类别n:\n \
\n \
### 二级类别m:\n \
- 三级类别1:[简短的描述]\n \
- 三级类别2:\n \
- 三级类别n:\n \
<|im_end|>\
'
#parts=['背景','过程','意义']
brace_map_instruction='我将给你一个文本，请先找出中心事件，然后分{}几个部分总结文本，请按下面格式输出。\n \
## 中心事件:[中心事件名称]\
### 部分1：[名称]\n\
 - \
 - \
### 部分2：\n\
 - \
 - \
### 部分n：\n\
 - \
 - \
<|im_end|>\
'
flow_map_instruction='我将给你一个文本，请按时间顺序将事件划分为若干个阶段(不超过5个),每个阶段包括若干个子事件，并按下面格式进行总结。\n \
## 阶段1:\n \
- 子事件1:[简短的描述]\n \
- 子事件2:\n \
- 子事件m:\n \
## 阶段2:\n \
- 子事件1:[简短的描述]\n \
- 子事件2:\n \
- 子事件m:\n \
## 阶段n:\n \
- 子事件1:[简短的描述]\n \
- 子事件2:\n \
- 子事件m:\n \
<|im_end|>\
'
multi_flow_map_instruction='我将给你一个描述了一个事件的文本，请找出事件名称，并简短地总结事件的原因和影响，按下面格式输出。\n \
## 事件:[事件名称]\n \
### 原因：\
- 原因1 \
- 原因2 \
- 原因n \
### 影响：\
- 影响1 \
- 影响2 \
- 影响m \
<|im_end|>\
'
## 各种类型图的指令
instruction_dict={
    'bubble_map':bubble_map_instruction,
    'double_bubble_map':double_bubble_map_instruction,
    'tree_map':tree_map_instruction,
    'brace_map':brace_map_instruction,
    'flow_map':flow_map_instruction,
    'multi_flow_map':multi_flow_map_instruction
}
def read_txt(txt_path):
    with open(txt_path,'r',encoding='utf-8') as f:
        input=f.read()
    return input

def get_llm_respond(input_or_txt_path,map_type,parts_list=None):
    """
    input_or_txt_path:输入内容或路径
    map_type:图类型
    parts_list:组织图需要给子标题，否则效果不好
    """
    if '.txt' in input_or_txt_path:
        input=read_txt(input_or_txt_path)
    else:
        input=input_or_txt_path
        
    instruction=instruction_dict[map_type]
    if map_type=='brace_map':
        instruction=instruction.format('、'.join(parts_list))
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction, # instruction
            input,
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)
    outputs = model.generate(**inputs, 
                             streamer = text_streamer, 
                             max_new_tokens = 2048,
                             )
    respond=tokenizer.batch_decode(outputs)[0].split('### 回答:\n')[-1]
    save_llm_respond(respond,f'LLM_respond_for_{map_type}.txt')  


## 使用封装后的函数得到每种图LLM输出

In [ ]:
path1='/kaggle/input/test-input/1_.txt'
path2='/kaggle/input/test-input/2_.txt'
path3='/kaggle/input/test-input/3_.txt'
path4='/kaggle/input/test-input/4_.txt'

In [32]:
## 气泡图
map_type='bubble_map'
args={
    'input_or_txt_path':path1,
    'map_type':map_type,
}
get_llm_respond(**args)

下面是描述任务的指令，并配有提供进一步上下文的输入。请按指令要求给出合适的回答。

### 指令:
我将给你一个文本，请找出文本的中心事件，并根据文本总结中心事件的相关属性和简短地描述，并按下面格式输出。
 ## 中心事件:[中心事件名称]
 ### 属性1:[属性名]
 #### 属性描述:[简短的描述]
### 属性2:[属性名]
 #### 属性描述:[简短的描述]
### 属性n:[属性名]
 #### 属性描述:[简短的描述]
<|im_end|>

### 输入:
兴建铁路作为一种官方政策是在新政之前就确立的，对近代中国的铁路建设起到了积极作用。自甲午战争结束到1911年，清王朝共建铁路4936公里，占近代中国所修铁路将近40%，而其中绝大部分是在新政时期建设而成。但主要依靠外资资助修筑的就长达4029公里，约占总长度的81%。但是作为回报，清政府不得不把铁路的管理权、用人权、稽核权、购料权拱手让给外国的借款公司，这就导致了中国利权的严重流失。即便如此，李鸿章在世时还是不无遗憾地说道：“目下经费难筹，必借洋款。”

1903年12月2日，满清政府颁布了《铁路简明章程》，规定“无论华人、洋人”均可向“督抚衙门递呈请办铁路”，而且包括“干路或枝路”，华人投资50万两以上实有成效者，朝廷还要“专折请旨给予优奖”。各省绅商随之掀起规模巨大的收回路权运动，这种行动在朝野上下产生了更大的共鸣。粤汉铁路于1900年承包给美国永兴公司建造，湖南、湖北和广东三省绅士以美国商人违反合同为由，要求收回粤汉铁路筑路权，转而自己修筑，此举得到张之洞的鼎力支持，湖南士绅王先谦从中周旋，终于在1905年8月以赔款675万美元为代价，赎回原先由美国公司发行的股票，废除了与美商签订的建设粤汉铁路的合同。在这个成功案例的鼓舞下，浙江、江苏、山东、直隶、吉林、云南等省收回路权的工作也先后付诸实施，要求赎回沪宁铁路、苏甬杭铁路、广九铁路承办权的社会呼声越来越高，并实际收回了一批筑路权，铁路商办运动进入高潮时期。在此后4年间，全国在15个省创设了18个大型铁路公司，其中17个是商办、官督商办或官商合办的。

清末商办铁路虽然资本匮乏，但是在国人爱国主义的热情鼓舞之下，均设置了严格的受款规定，坚决拒绝外资、不招洋股、不借洋债，故而必须进行大量的民间融资。铁路建设费用极大，一时间又筹不到足够资金，于

In [19]:
## 双泡图
map_type='double_bubble_map'
args={
    'input_or_txt_path':path4,
    'map_type':map_type,
}
get_llm_respond(**args)

下面是描述任务的指令，并配有提供进一步上下文的输入。请按指令要求给出合适的回答。

### 指令:
我将给你一个用于比较两个事物的文本，请找出两个事物，并根据下面格式总结两个事物的共同点和不同点。
 ## 比较对象: [对象1]，[对象2]
 ### 相同点:
 #### 相同点1:[简短的描述]
#### 相同点2:
 #### 相同点n:
 ### 对象1独有的特点:
#### 特点1:[简短的描述]
 #### 特点2:
 #### 特点n:
 ### 对象2独有的特点:
#### 特点1:[简短的描述]
 #### 特点2:
 #### 特点n:
 <|im_end|>

### 输入:
根据其细胞的基本结构，可以将所有活生物体分为两组之一：原核生物和真核生物。原核生物是由缺乏细胞核或任何膜包裹的细胞器的细胞组成的生物。真核生物是由具有膜结合核的细胞组成的生物，该核保留遗传物质以及膜结合的细胞器。

了解细胞和细胞膜

该电池是我们现代的生活和居住的东西定义的一个基本组成部分。细胞被认为是生命的基本组成部分，并用于难以理解的“活”的定义。

细胞使化学过程保持整齐和分隔，因此单个细胞过程不会干扰其他细胞，并且细胞可以进行代谢，繁殖等业务。为此，细胞成分被包裹在膜中，该膜可作为外部屏障世界和细胞的内部化学。的细胞膜为选择性屏障，这意味着它可以让某些化学品和其他人进行。这样做可以维持细胞存活所需的化学平衡。

细胞膜通过三种方式调节化学物质在细胞内外的穿越：

扩散（溶质分子使浓度最小化并因此从较高浓度的区域移向较低浓度的区域直到浓度相等的趋势）
渗透（溶剂跨过选择性边界移动，以平衡无法跨边界移动的溶质的浓度）
选择性运输（通过膜通道和膜泵）
原核生物:

原核生物是由缺乏细胞核或任何膜包裹的细胞器的细胞组成的生物。这意味着原核生物中的遗传物质DNA未结合在核内。此外，与原核生物相比，DNA在原核生物中的结构更少：在原核生物中，DNA是一个单环，而在真核生物中，DNA被组织成染色体。大多数原核生物仅由单个细胞（单细胞）组成，但也有少数由细胞集合（多细胞）组成。

科学家将原核生物分为细菌和古细菌两大类。食品中发现了一些细菌，包括大肠杆菌，沙门氏菌和李斯特菌，它们可能导致疾病。1 其他实际上有助于人体消化和其他功能。2 古细菌被发现是一种独特的生命形式，能够在热液

In [41]:
## 树状图
map_type='tree_map'
args={
    'input_or_txt_path':input3,
    'map_type':map_type,
}
get_llm_respond(**args)


下面是描述任务的指令，并配有提供进一步上下文的输入。请按指令要求给出合适的回答。

### 指令:
我将给你一个用于介绍某个事物相关类别的文本，请找出中心事物，并根据下面格式简短地总结事物的主要类别。
 ## 一级类别: [中心事物]
 
 ### 二级类别1:
 - 三级类别1:[简短的描述]
 - 三级类别2:
 - 三级类别n:
 
 ### 二级类别2:
 - 三级类别1:[简短的描述]
 - 三级类别2:
 - 三级类别n:
 
 ### 二级类别m:
 - 三级类别1:[简短的描述]
 - 三级类别2:
 - 三级类别n:
 <|im_end|>

### 输入:

低纬度气候：

　　低纬度的气候主要受赤道气团和热带气团所控制。影响气候的主要环流系统有赤道辐合带、瓦克环流、信风、赤道西风、热带气旋和副热带高压。全年地气系统的辐射差额是入超的，因此气温全年皆高，最冷月平均气温在15～18℃以上，全年水分可能蒸散量在130cm以上。本带可分为5个气候型，其中热带干旱与半干旱气候型又可划分为3个亚型。

赤道多雨气候

　　分布于赤道及其南、北5°～10°以内，宽窄不一，主要分布在非洲扎伊尔河流域、南美亚马孙河流域和亚洲与大洋洲间的从苏门答腊岛到伊里安岛一带。这里全年正午太阳高度都很大，因此长夏无冬，各月平均气温在25°28℃，年平均气温在26℃左右。气温年较差一般小于3℃，日较差可达6～12℃。由于全年皆在赤道气团控制下，风力微弱，以辐合上升气流为主，多雷阵雨，因此全年多雨，无干季，年降水量在2000mm以上，最少月在60mm以上。但降水量的年际变化很大，这与赤道辐合带位置的变动有关。

热带海洋性气候

　　分布在南北纬10°～25°信风带大陆东岸及热带海洋中的若干岛屿上。这里正当迎风海岸，全年盛行热带海洋气团，气候具有海洋性，最热月平均气温在28℃左右，最冷月平均气温在18～25℃之间，气温年较差、日较差皆小。由于东风（信风）带来湿热的海洋气团，所以除对流雨、热带气旋雨外，还多地形雨，降水量充沛。年降水量在1000mm以上，一般以5～10月较集中，无明显变化。

热带干湿季气候

　　大致分布在南北半球5°～25°之间。这里当正午太阳高度较小时，位于信风带下，受热带大陆气团控制，盛行下沉气流，是为干季。当正午太阳高度较大时，赤道辐合带移来，有潮湿的辐合

In [43]:
## 组织图
map_type='brace_map'
## 组织图需要另外设置子标题，否则效果不佳
parts=['背景','过程','意义']
args={
    'input_or_txt_path':path1,
    'map_type':map_type,
    'parts_list':parts
}
get_llm_respond(**args)

下面是描述任务的指令，并配有提供进一步上下文的输入。请按指令要求给出合适的回答。

### 指令:
我将给你一个文本，请先找出中心事件，然后分背景、过程、意义几个部分总结文本，请按下面格式输出。
 ## 中心事件:[中心事件名称]### 部分1：[名称]
 -  - ### 部分2：
 -  - ### 部分n：
 -  - <|im_end|>

### 输入:
兴建铁路作为一种官方政策是在新政之前就确立的，对近代中国的铁路建设起到了积极作用。自甲午战争结束到1911年，清王朝共建铁路4936公里，占近代中国所修铁路将近40%，而其中绝大部分是在新政时期建设而成。但主要依靠外资资助修筑的就长达4029公里，约占总长度的81%。但是作为回报，清政府不得不把铁路的管理权、用人权、稽核权、购料权拱手让给外国的借款公司，这就导致了中国利权的严重流失。即便如此，李鸿章在世时还是不无遗憾地说道：“目下经费难筹，必借洋款。”

1903年12月2日，满清政府颁布了《铁路简明章程》，规定“无论华人、洋人”均可向“督抚衙门递呈请办铁路”，而且包括“干路或枝路”，华人投资50万两以上实有成效者，朝廷还要“专折请旨给予优奖”。各省绅商随之掀起规模巨大的收回路权运动，这种行动在朝野上下产生了更大的共鸣。粤汉铁路于1900年承包给美国永兴公司建造，湖南、湖北和广东三省绅士以美国商人违反合同为由，要求收回粤汉铁路筑路权，转而自己修筑，此举得到张之洞的鼎力支持，湖南士绅王先谦从中周旋，终于在1905年8月以赔款675万美元为代价，赎回原先由美国公司发行的股票，废除了与美商签订的建设粤汉铁路的合同。在这个成功案例的鼓舞下，浙江、江苏、山东、直隶、吉林、云南等省收回路权的工作也先后付诸实施，要求赎回沪宁铁路、苏甬杭铁路、广九铁路承办权的社会呼声越来越高，并实际收回了一批筑路权，铁路商办运动进入高潮时期。在此后4年间，全国在15个省创设了18个大型铁路公司，其中17个是商办、官督商办或官商合办的。

清末商办铁路虽然资本匮乏，但是在国人爱国主义的热情鼓舞之下，均设置了严格的受款规定，坚决拒绝外资、不招洋股、不借洋债，故而必须进行大量的民间融资。铁路建设费用极大，一时间又筹不到足够资金，于是政府不得不出面协助，一时“租股、派股”盛极全国各地。湖北咨议局全体会议通过决议，后来又进一步提出拒款方案十项，其

In [31]:
## 流程图
map_type='flow_map'
args={
    'input_or_txt_path':path2,
    'map_type':map_type,
}
get_llm_respond(**args)

下面是描述任务的指令，并配有提供进一步上下文的输入。请按指令要求给出合适的回答。

### 指令:
我将给你一个文本，请按时间顺序将事件划分为若干个阶段(不超过5个),每个阶段包括若干个子事件，并按下面格式进行总结。
 ## 阶段1:
 - 子事件1:[简短的描述]
 - 子事件2:
 - 子事件m:
 ## 阶段2:
 - 子事件1:
 - 子事件2:
 - 子事件m:
 ## 阶段n:
 - 子事件1:
 - 子事件2:
 - 子事件m:
 <|im_end|>

### 输入:
第十三回 秦可卿死封龙禁尉 王熙凤协理宁国府

    话说凤姐儿自贾琏送黛玉往扬州去后，心中实在无趣，每到晚间，不过和平儿说笑一回，就胡乱睡了。

    这日夜间，正和平儿灯下拥炉倦绣，早命浓薰绣被，二人睡下，屈指算行程该到何处，不知不觉已交三鼓。平儿已睡熟了。凤姐方觉星眼微朦，恍惚只见秦氏从外走来，含笑说道：“婶子好睡！我今日回去，你也不送我一程。因娘儿们素日相好，我舍不得婶子，故来别你一别。还有一件心愿未了，非告诉婶子，别人未必中用。”

    凤姐听了，恍惚问道：“有何心愿？你只管托我就是了。”秦氏道：“婶婶，你是个脂粉队里的英雄，连那些束带顶冠的男子也不能过你，你如何连两句俗语也不晓得？常言。月满则亏，水满则溢&#039;，又道是。登高必跌重&#039;。如今我们家赫赫扬扬，已将百载，一日倘或乐极悲生，若应了那句。树倒猢狲散&#039;的俗语，岂不虚称了一世的诗书旧族了！”凤姐听了此话，心胸大快，十分敬畏，忙问道：这话虑的极是，但有何法可以永保无虞？”秦氏冷笑道：“婶子好痴也。否极泰来，荣辱自古周而复始，岂人力能可保常的。但如今能于荣时筹画下将来衰时的世业，亦可谓常保永全了。

    即如今日诸事都妥，只有两件未妥，若把此事如此一行，则后日可保永全了。

    凤姐便问何事。秦氏道：“目今祖茔虽四时祭祀，只是无一定的钱粮，第二，家塾虽立，无一定的供给。依我想来，如今盛时固不缺祭祀供给，但将来败落之时，此二项有何出处？莫若依我定见，趁今日富贵，将祖茔附近多置田庄房舍地亩，以备祭祀供给之费皆出自此处，将家塾亦设于此。合同族中长幼，大家定了则例，日后按房掌管这一年的地亩，钱粮，祭祀，供给之事。如此周流，又无争竞，亦不有典卖诸弊。便是有了罪，凡物可入官，这祭祀产业连官也不入的

In [37]:
map_type='multi_flow_map'
args={
    'input_or_txt_path':path1,
    'map_type':map_type,
}
get_llm_respond(**args)

下面是描述任务的指令，并配有提供进一步上下文的输入。请按指令要求给出合适的回答。

### 指令:
我将给你一个描述了一个事件的文本，请找出事件名称，并简短地总结事件的原因和影响，按下面格式输出。
 ## 事件:[事件名称]
 ### 原因：- 原因1 - 原因2 - 原因n ### 影响：- 影响1 - 影响2 - 影响m <|im_end|>

### 输入:
兴建铁路作为一种官方政策是在新政之前就确立的，对近代中国的铁路建设起到了积极作用。自甲午战争结束到1911年，清王朝共建铁路4936公里，占近代中国所修铁路将近40%，而其中绝大部分是在新政时期建设而成。但主要依靠外资资助修筑的就长达4029公里，约占总长度的81%。但是作为回报，清政府不得不把铁路的管理权、用人权、稽核权、购料权拱手让给外国的借款公司，这就导致了中国利权的严重流失。即便如此，李鸿章在世时还是不无遗憾地说道：“目下经费难筹，必借洋款。”

1903年12月2日，满清政府颁布了《铁路简明章程》，规定“无论华人、洋人”均可向“督抚衙门递呈请办铁路”，而且包括“干路或枝路”，华人投资50万两以上实有成效者，朝廷还要“专折请旨给予优奖”。各省绅商随之掀起规模巨大的收回路权运动，这种行动在朝野上下产生了更大的共鸣。粤汉铁路于1900年承包给美国永兴公司建造，湖南、湖北和广东三省绅士以美国商人违反合同为由，要求收回粤汉铁路筑路权，转而自己修筑，此举得到张之洞的鼎力支持，湖南士绅王先谦从中周旋，终于在1905年8月以赔款675万美元为代价，赎回原先由美国公司发行的股票，废除了与美商签订的建设粤汉铁路的合同。在这个成功案例的鼓舞下，浙江、江苏、山东、直隶、吉林、云南等省收回路权的工作也先后付诸实施，要求赎回沪宁铁路、苏甬杭铁路、广九铁路承办权的社会呼声越来越高，并实际收回了一批筑路权，铁路商办运动进入高潮时期。在此后4年间，全国在15个省创设了18个大型铁路公司，其中17个是商办、官督商办或官商合办的。

清末商办铁路虽然资本匮乏，但是在国人爱国主义的热情鼓舞之下，均设置了严格的受款规定，坚决拒绝外资、不招洋股、不借洋债，故而必须进行大量的民间融资。铁路建设费用极大，一时间又筹不到足够资金，于是政府不得不出面协助，一时“租股、派股”盛极全国各地。湖北咨议局全体会议通过决议，后来又进一步提出拒款方案十项，其